In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from get_decision_variable_map import get_decision_variable_map
from get_case_outputs_all_models import get_case_outputs_all_models
from get_printable_resource_names import get_printable_resource_names

In [2]:
def percent_change_df(df, normalize_model):
    norm = df.copy()
    norm['CEM'] = df['CEM'].astype(float) / df[normalize_model].astype(float)
    norm['PF'] = df['PF'].astype(float) / df[normalize_model].astype(float)
    norm['DLAC-p'] = df['DLAC-p'].astype(float) / df[normalize_model].astype(float)
    norm['DLAC-i'] = df['DLAC-i'].astype(float) / df[normalize_model].astype(float)
    norm['SLAC'] = df['SLAC'].astype(float) / df[normalize_model].astype(float)

    change_df = norm.copy()
    change_df['CEM'] = norm['CEM'] - 1
    change_df['PF'] = norm['PF'] - 1
    change_df['DLAC-p'] = norm['DLAC-p'] - 1
    change_df['DLAC-i'] = norm['DLAC-i'] - 1
    change_df['SLAC'] = norm['SLAC'] - 1

    percent_change_df = change_df.copy()
    percent_change_df['CEM'] = change_df['CEM'] * 100
    percent_change_df['PF'] = change_df['PF'] * 100
    percent_change_df['DLAC-p'] = change_df['DLAC-p'] * 100
    percent_change_df['DLAC-i'] = change_df['DLAC-i'] * 100
    percent_change_df['SLAC'] = change_df['SLAC'] * 100

    # remove decimal point
    # percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']] = \
    #                     percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']].round(2)
    # percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']] = \
    #     percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']].astype(str) + '%'

    return percent_change_df


In [3]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

policy_types = [
                'pf',
                'dlac-p',
                'dlac-i',
                'slac',
]

In [4]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [5]:
plots_path = os.path.join(current_dir, 'plots') + "/"
pdf_path = os.path.join(current_dir, 'pdf_tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
jpg_path = os.path.join(current_dir, 'jpg') + "/"
if not os.path.exists(plots_path):
    os.makedirs(plots_path)
if not os.path.exists(pdf_path):
    os.makedirs(pdf_path)
if not os.path.exists(latex_path):
    os.makedirs(latex_path)
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
if not os.path.exists(jpg_path):
    os.makedirs(jpg_path)


In [6]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

params_names = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
          'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost', 'EnergyRevenue', 
          'OperatingReserveRevenue', 'OperatingRegulationRevenue']

param_print_names = params_names

for i in range(len(param_print_names)):
    param_print_names[i] = param_print_names[i].replace("_", " ")

# economic_cost_params = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
#         'Fuel_cost', 'Var_OM_cost_in', 'StartCost']

system_costs = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
        'Fuel_cost', 'Var_OM_cost_in', 'StartCost']

economic_cost_params = system_costs + ['Charge_cost']

economic_revenue_params = ['EnergyRevenue', 'OperatingReserveRevenue', 'OperatingRegulationRevenue']

In [7]:
# get load data

load_data_raw = pd.read_csv(cem_path + '\\Thermal_Base\\' + 'system\\' + 'Demand_data.csv')
load_data = load_data_raw['Demand_MW_z1']

In [8]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [9]:
unit_profit_dic = {case: None for case in case_names}
percentage_profits_dic = {case: None for case in case_names}
consumer_payments_dic = {case: None for case in case_names}
consumer_payments_ratio_dic = {case: None for case in case_names}
total_investment_costs_dic = {case: None for case in case_names}
total_operating_costs_dic = {case: None for case in case_names}
total_system_costs_dic = {case: None for case in case_names} 
total_inv_op_costs_dic = {case: None for case in case_names}
total_objective_dic = {case: None for case in case_names}
nse_costs_dic = {case: None for case in case_names}
total_nse_dic = {case: None for case in case_names}
total_unmet_rsv_dic = {case: None for case in case_names}
total_nse_cost_dic = {case: None for case in case_names}
total_unmet_rsv_cost_dic = {case: None for case in case_names}
total_fixed_om_costs_dic = {case: None for case in case_names}
total_var_om_out_costs_dic = {case: None for case in case_names}
total_fuel_costs_dic = {case: None for case in case_names}
total_var_om_in_costs_dic = {case: None for case in case_names}
total_start_costs_dic = {case: None for case in case_names}
total_charge_costs_dic = {case: None for case in case_names}

In [10]:
for case_name in case_names:

    print('Case Name: ' + case_name + '\n')

    # load generator characteristics from resources folder
    thermal_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Thermal.csv')
    vre_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Vre.csv')
    storage_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Storage.csv')

    # combine all resources to dfGen
    dfGen = pd.concat([thermal_dfGen, vre_dfGen, storage_dfGen], ignore_index=True)

    gen_capacity_gw = dfGen['Existing_Cap_MW'] / ModelScalingFactor
    gen_capacity_mw = dfGen['Existing_Cap_MW']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh'] / ModelScalingFactor

    resource_list = dfGen['Resource']

    # solar and wind id
    solar_id = dfGen[dfGen['Resource'] == 'Utility PV - Class 1'].index[0]
    wind_id = dfGen[dfGen['Resource'] == 'Land-Based Wind - Class 1 - Technology 1'].index[0]

    cem_case = 'CEM_' + case_name + '_ABB\\'
    pf_case = 'PF_' + case_name + '\\'
    dlac_case = 'DLAC_' + case_name + '\\'
    dlac_imperfect_case = 'DLAC_imperfect_' + case_name + '\\'
    slac_case = 'SLAC_' + case_name + '\\'

    cem_results_path = cem_path + '\\' + case_name + '\\' + 'results\\'
    pf_results_path = policies_path + '\\' + case_name + '\\' + 'results_pf\\'
    dlac_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-p\\'
    dlac_imperfect_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-i\\'
    slac_results_path = policies_path + '\\' + case_name + '\\' + 'results_slac\\'

    cem_raw = pd.read_csv(cem_results_path + 'NetRevenue.csv')
    pf_raw = pd.read_csv(pf_results_path + 'NetRevenue.csv')
    dlac_raw = pd.read_csv(dlac_results_path + 'NetRevenue.csv')
    dlac_imperfect_raw = pd.read_csv(dlac_imperfect_results_path + 'NetRevenue.csv')
    slac_raw = pd.read_csv(slac_results_path + 'NetRevenue.csv')

    cem_raw_prices = pd.read_csv(cem_results_path + 'prices.csv')
    pf_raw_prices = pd.read_csv(pf_results_path + 'price_electricity.csv', header=None)
    dlac_raw_prices = pd.read_csv(dlac_results_path + 'price_electricity.csv', header=None)
    dlac_imperfect_raw_prices = pd.read_csv(dlac_imperfect_results_path + 'price_electricity.csv', header=None)
    slac_raw_prices = pd.read_csv(slac_results_path + 'price_electricity.csv', header=None)

    column_names = cem_raw.columns[1:]
    ### get cem dataframe of only non-trivial resources
    cem_raw_upd = cem_raw[cem_raw['Resource'].isin(resource_list)]
    # make sure the order of the resources is the same in dfGen and cem_raw_upd
    cem_raw_upd = cem_raw_upd.set_index('Resource').reindex(resource_list).reset_index()


    gen_print_names = resource_list
    # print(gen_print_names)

    param_print_names = params_names

    for i in range(len(param_print_names)):
        param_print_names[i] = param_print_names[i].replace("_", " ")

    # get cem total nse and unmet rsv and total cem nse and unmet rsv costs
    cem_costs_genx = pd.read_csv(cem_results_path + 'costs.csv')
    cem_sum_ur_cost = cem_costs_genx[cem_costs_genx['Costs'] == 'cUnmetRsv']['Total'].values[0]
    cem_total_unmet_rsv = cem_sum_ur_cost / 1000
    cem_total_nse_cost = cem_costs_genx[cem_costs_genx['Costs'] == 'cNSE']['Total'].values[0]
    cem_total_nse = cem_total_nse_cost / 5000

    cem_nse, nse_policies_dic, col_names = get_case_outputs_all_models(cem_path, policies_path, case_name, 'non-served energy', policy_types)
    # load in timeseries of nse and unmet rsv from LACs
    # cem_nse = pd.read_csv(cem_results_path + 'nse.csv')
    # pf_nse = pd.read_csv(pf_results_path + 'zone_nse.csv', header=None)
    # dlac_nse = pd.read_csv(dlac_results_path + 'zone_nse.csv', header=None)
    # dlac_imperfect_nse = pd.read_csv(dlac_imperfect_results_path + 'zone_nse.csv', header=None)
    # slac_nse = pd.read_csv(slac_results_path + 'zone_nse.csv', header=None)
    pf_nse = nse_policies_dic['pf']
    dlac_nse = nse_policies_dic['dlac-p']
    dlac_imperfect_nse = nse_policies_dic['dlac-i']
    slac_nse = nse_policies_dic['slac']

    dlac_ur = pd.read_csv(dlac_results_path + 'zone_unmet_rsv.csv', header=None)
    pf_ur = pd.read_csv(pf_results_path + 'zone_unmet_rsv.csv', header=None)
    dlac_imperfect_ur = pd.read_csv(dlac_imperfect_results_path + 'zone_unmet_rsv.csv', header=None)
    slac_ur = pd.read_csv(slac_results_path + 'zone_unmet_rsv.csv', header=None)

    pf_nse_total = pf_nse.sum(axis=0)
    dlac_nse_total = dlac_nse.sum(axis=0)
    dlac_imperfect_nse_total = dlac_imperfect_nse.sum(axis=0)
    slac_nse_total = slac_nse.sum(axis=0)

    pf_ur_total = pf_ur.sum(axis=0)
    dlac_ur_total = dlac_ur.sum(axis=0)
    dlac_imperfect_ur_total = dlac_imperfect_ur.sum(axis=0)
    slac_ur_total = slac_ur.sum(axis=0)

    # print(dlac_nse_total)

    nse_totals = {
        'CEM': f"{cem_total_nse:.4e}",
        'PF': f"{pf_nse_total[0]:.4e}",
        'DLAC-p': f"{dlac_nse_total[0]:.4e}",
        'DLAC-i': f"{dlac_imperfect_nse_total[0]:.4e}",
        'SLAC': f"{slac_nse_total[0]:.4e}"
    }

    unmet_rsv_totals = {
        'CEM': f"{cem_total_unmet_rsv:.4e}",
        'PF': f"{pf_ur_total[0]:.4e}",
        'DLAC-p': f"{dlac_ur_total[0]:.4e}",
        'DLAC-i': f"{dlac_imperfect_ur_total[0]:.4e}",
        'SLAC': f"{slac_ur_total[0]:.4e}"
    }


    total_nse_dic[case_name] = nse_totals
    total_unmet_rsv_dic[case_name] = unmet_rsv_totals

    # calculate reg and rsv revenue sums
    cem_reg_revenue_sum = cem_raw_upd['OperatingRegulationRevenue'].sum()
    cem_rsv_revenue_sum = cem_raw_upd['OperatingReserveRevenue'].sum()

    pf_reg_revenue_sum = pf_raw['OperatingRegulationRevenue'].sum()
    pf_rsv_revenue_sum = pf_raw['OperatingReserveRevenue'].sum()

    dlac_reg_revenue_sum = dlac_raw['OperatingRegulationRevenue'].sum()
    dlac_rsv_revenue_sum = dlac_raw['OperatingReserveRevenue'].sum()

    dlac_imperfect_reg_revenue_sum = dlac_imperfect_raw['OperatingRegulationRevenue'].sum()
    dlac_imperfect_rsv_revenue_sum = dlac_imperfect_raw['OperatingReserveRevenue'].sum()

    slac_reg_revenue_sum = slac_raw['OperatingRegulationRevenue'].sum()
    slac_rsv_revenue_sum = slac_raw['OperatingReserveRevenue'].sum()



    # load unmet reserves cost information
    pf_ur_cost = pd.read_csv(pf_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_ur_cost = pd.read_csv(dlac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_imperfect_ur_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    slac_ur_cost = pd.read_csv(slac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)

    # if there are more columns than rows, transpose the dataframe
    if pf_ur_cost.shape[0] < pf_ur_cost.shape[1]:
        pf_ur_cost = pf_ur_cost.T
    if dlac_ur_cost.shape[0] < dlac_ur_cost.shape[1]:
        dlac_ur_cost = dlac_ur_cost.T
    if dlac_imperfect_ur_cost.shape[0] < dlac_imperfect_ur_cost.shape[1]:
        dlac_imperfect_ur_cost = dlac_imperfect_ur_cost.T
    if slac_ur_cost.shape[0] < slac_ur_cost.shape[1]:
        slac_ur_cost = slac_ur_cost.T


    pf_nse_cost = pd.read_csv(pf_results_path + 'revenue_nse_cost.csv', header=None)
    dlac_nse_cost = pd.read_csv(dlac_results_path + 'revenue_nse_cost.csv', header=None)
    dlac_imperfect_nse_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_nse_cost.csv', header=None)
    slac_nse_cost = pd.read_csv(slac_results_path + 'revenue_nse_cost.csv', header=None)

    # if there are more columns than rows, transpose the dataframe
    if pf_nse_cost.shape[0] < pf_nse_cost.shape[1]:
        pf_nse_cost = pf_nse_cost.T
    if dlac_nse_cost.shape[0] < dlac_nse_cost.shape[1]:
        dlac_nse_cost = dlac_nse_cost.T
    if dlac_imperfect_nse_cost.shape[0] < dlac_imperfect_nse_cost.shape[1]:
        dlac_imperfect_nse_cost = dlac_imperfect_nse_cost.T
    if slac_nse_cost.shape[0] < slac_nse_cost.shape[1]:
        slac_nse_cost = slac_nse_cost.T

    pf_sum_nse_cost = pf_nse_cost.sum(axis=0)[0]
    dlac_sum_nse_cost = dlac_nse_cost.sum(axis=0)[0]
    dlac_imperfect_sum_nse_cost = dlac_imperfect_nse_cost.sum(axis=0)[0]
    slac_sum_nse_cost = slac_nse_cost.sum(axis=0)[0]

    pf_sum_ur_cost = pf_ur_cost.sum(axis=0)[0]
    dlac_sum_ur_cost = dlac_ur_cost.sum(axis=0)[0]
    dlac_imperfect_sum_ur_cost = dlac_imperfect_ur_cost.sum(axis=0)[0]
    slac_sum_ur_cost = slac_ur_cost.sum(axis=0)[0]

    total_nse_cost_dic[case_name] = {
        'CEM': f"{cem_total_nse_cost:.4e}",
        'PF': f"{pf_sum_nse_cost:.4e}",
        'DLAC-p': f"{dlac_sum_nse_cost:.4e}",
        'DLAC-i': f"{dlac_imperfect_sum_nse_cost:.4e}",
        'SLAC': f"{slac_sum_nse_cost:.4e}"
    }

    total_unmet_rsv_cost_dic[case_name] = {
        'CEM': f"{cem_sum_ur_cost:.4e}",
        'PF': f"{pf_sum_ur_cost:.4e}",
        'DLAC-p': f"{dlac_sum_ur_cost:.4e}",
        'DLAC-i': f"{dlac_imperfect_sum_ur_cost:.4e}",
        'SLAC': f"{slac_sum_ur_cost:.2e}"
    }

    cem_prices = cem_raw_prices['1']
    pf_prices = pf_raw_prices[0]
    dlac_prices = dlac_raw_prices[0]
    dlac_imperfect_prices = dlac_imperfect_raw_prices[0]
    slac_prices = slac_raw_prices[0]

    cem_costs = cem_raw_upd[economic_cost_params]
    pf_costs = pf_raw[economic_cost_params]
    dlac_costs = dlac_raw[economic_cost_params]
    dlac_imperfect_costs = dlac_imperfect_raw[economic_cost_params]
    slac_costs = slac_raw[economic_cost_params]

    cem_economics = cem_raw_upd[economic_cost_params + economic_revenue_params]
    pf_economics = pf_raw[economic_cost_params + economic_revenue_params]
    dlac_economics = dlac_raw[economic_cost_params + economic_revenue_params]
    dlac_imperfect_economics = dlac_imperfect_raw[economic_cost_params + economic_revenue_params]
    slac_economics = slac_raw[economic_cost_params + economic_revenue_params]

    cem_economics_copy = cem_economics.copy()
    pf_economics_copy = pf_economics.copy()
    dlac_economics_copy = dlac_economics.copy()
    dlac_imperfect_economics_copy = dlac_imperfect_economics.copy()
    slac_economics_copy = slac_economics.copy()

    cem_economics_copy[economic_cost_params] = cem_economics_copy[economic_cost_params] * -1
    pf_economics_copy[economic_cost_params] = pf_economics_copy[economic_cost_params] * -1
    dlac_economics_copy[economic_cost_params] = dlac_economics_copy[economic_cost_params] * -1
    dlac_imperfect_economics_copy[economic_cost_params] = dlac_imperfect_economics_copy[economic_cost_params] * -1
    slac_economics_copy[economic_cost_params] = slac_economics_copy[economic_cost_params] * -1

    cem_profit = cem_economics_copy.sum(axis=1)
    pf_profit = pf_economics_copy.sum(axis=1)
    dlac_profit = dlac_economics_copy.sum(axis=1)
    dlac_imperfect_profit = dlac_imperfect_economics_copy.sum(axis=1)
    slac_profit = slac_economics_copy.sum(axis=1)

    cem_economics_scaled = cem_economics_copy / ModelScalingFactor**2
    pf_economics_scaled = pf_economics_copy / ModelScalingFactor**2
    dlac_economics_scaled = dlac_economics_copy / ModelScalingFactor**2
    dlac_imperfect_economics_scaled = dlac_imperfect_economics_copy / ModelScalingFactor**2
    slac_economics_scaled = slac_economics_copy / ModelScalingFactor**2

    cem_scaled_profit = cem_economics_scaled.sum(axis=1)
    pf_scaled_profit = pf_economics_scaled.sum(axis=1)
    dlac_scaled_profit = dlac_economics_scaled.sum(axis=1)
    dlac_imperfect_scaled_profit = dlac_imperfect_economics_scaled.sum(axis=1)
    slac_scaled_profit = slac_economics_scaled.sum(axis=1)


    # define system costs for total system and objective calculations
    cem_system_costs = cem_raw_upd[system_costs]
    pf_system_costs = pf_raw[system_costs]
    dlac_system_costs = dlac_raw[system_costs]
    dlac_imperfect_system_costs = dlac_imperfect_raw[system_costs]
    slac_system_costs = slac_raw[system_costs]

    ###================================================================================================
    ### Total Investment Costs
    ###================================================================================================
    cem_mw_investment_costs = cem_raw['Inv_cost_MW'].sum()
    cem_mwhour_investment_costs = cem_raw['Inv_cost_MWh'].sum()

    pf_mw_investment_costs = pf_raw['Inv_cost_MW'].sum()
    pf_mwhour_investment_costs = pf_raw['Inv_cost_MWh'].sum()

    dlac_mw_investment_costs = dlac_raw['Inv_cost_MW'].sum()
    dlac_mwhour_investment_costs = dlac_raw['Inv_cost_MWh'].sum()

    dlac_imperfect_mw_investment_costs = dlac_imperfect_raw['Inv_cost_MW'].sum()
    dlac_imperfect_mwhour_investment_costs = dlac_imperfect_raw['Inv_cost_MWh'].sum()

    slac_mw_investment_costs = slac_raw['Inv_cost_MW'].sum()
    slac_mwhour_investment_costs = slac_raw['Inv_cost_MWh'].sum()

    cem_total_investment_costs = cem_mw_investment_costs + cem_mwhour_investment_costs
    pf_total_investment_costs = pf_mw_investment_costs + pf_mwhour_investment_costs
    dlac_total_investment_costs = dlac_mw_investment_costs + dlac_mwhour_investment_costs
    dlac_imperfect_total_investment_costs = dlac_imperfect_mw_investment_costs + dlac_imperfect_mwhour_investment_costs
    slac_total_investment_costs = slac_mw_investment_costs + slac_mwhour_investment_costs

    # cem_total_investment_costs_sci = f"{cem_total_investment_costs:.4e}"
    # pf_total_investment_costs_sci = f"{pf_total_investment_costs:.4e}"
    # dlac_total_investment_costs_sci = f"{dlac_total_investment_costs:.4e}"
    # dlac_imperfect_total_investment_costs_sci = f"{dlac_imperfect_total_investment_costs:.4e}"
    # slac_total_investment_costs_sci = f"{slac_total_investment_costs:.4e}"

    total_investment_costs = {
        'CEM': cem_total_investment_costs,
        'PF': pf_total_investment_costs,
        'DLAC-p': dlac_total_investment_costs,
        'DLAC-i': dlac_imperfect_total_investment_costs,
        'SLAC': slac_total_investment_costs
    }
    total_investment_costs_dic[case_name] = total_investment_costs


    ###================================================================================================
    ### Unit Profits
    ###================================================================================================
    
    cem_unit_profit_mw = cem_profit / gen_capacity_mw
    pf_unit_profit_mw = pf_profit / gen_capacity_mw
    dlac_unit_profit_mw = dlac_profit / gen_capacity_mw
    dlac_imperfect_unit_profit_mw = dlac_imperfect_profit / gen_capacity_mw
    slac_unit_profit_mw = slac_profit / gen_capacity_mw

    cem_unit_profit_mw_sci = cem_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    pf_unit_profit_mw_sci = pf_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_unit_profit_mw_sci = dlac_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_imperfect_unit_profit_mw_sci = dlac_imperfect_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    slac_unit_profit_mw_sci = slac_unit_profit_mw.apply(lambda x: f"{x:.4e}")

    cem_unit_profit_gw = cem_profit / gen_capacity_gw
    pf_unit_profit_gw = pf_profit / gen_capacity_gw
    dlac_unit_profit_gw = dlac_profit / gen_capacity_gw
    dlac_imperfect_unit_profit_gw = dlac_imperfect_profit / gen_capacity_gw
    slac_unit_profit_gw = slac_profit / gen_capacity_gw

    unit_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_unit_profit_mw,
        'PF': pf_unit_profit_mw,
        'DLAC-p': dlac_unit_profit_mw,
        'DLAC-i': dlac_imperfect_unit_profit_mw,
        'SLAC': slac_unit_profit_mw
    })

    # sorted_unit_profits = sorted(unit_profits.items(), key=lambda item: item[1], reverse=True)

    unit_profit_dic[case_name] = unit_profit_df

    ###================================================================================================
    ### Calculate consumer payments
    ###================================================================================================

    cem_consumer_payments = cem_prices * (load_data - cem_nse['1'])  # - nse?
    pf_consumer_payments = pf_prices * (load_data - pf_nse['1'])  # - nse?
    dlac_consumer_payments = dlac_prices * (load_data - dlac_nse['1'])  # - nse?
    dlac_imperfect_consumer_payments = dlac_imperfect_prices * (load_data - dlac_imperfect_nse['1'])  # - nse?
    slac_consumer_payments = slac_prices * (load_data - slac_nse['1'])  # - nse?

    sum_cem_cp = cem_consumer_payments.sum() + cem_reg_revenue_sum + cem_rsv_revenue_sum
    sum_pf_cp = pf_consumer_payments.sum() + pf_reg_revenue_sum + pf_rsv_revenue_sum
    sum_dlac_cp = dlac_consumer_payments.sum() + dlac_reg_revenue_sum + dlac_rsv_revenue_sum
    sum_dlac_imperfect_cp = dlac_imperfect_consumer_payments.sum() + \
                                    dlac_imperfect_reg_revenue_sum + dlac_imperfect_rsv_revenue_sum
    sum_slac_cp = slac_consumer_payments.sum()  + slac_reg_revenue_sum + slac_rsv_revenue_sum

    # # Convert sums to millions of dollars
    # sum_cem_cp_mil = sum_cem_cp / 1e6
    # sum_dlac_cp_mil = sum_dlac_cp / 1e6
    # sum_dlac_imperfect_cp_mil = sum_dlac_imperfect_cp / 1e6
    # sum_slac_cp_mil = sum_slac_cp / 1e6

    sum_cem_cp_sci = f"{sum_cem_cp:.4e}"
    sum_pf_cp_sci = f"{sum_pf_cp:.4e}"
    sum_dlac_cp_sci = f"{sum_dlac_cp:.4e}"
    sum_dlac_imperfect_cp_sci = f"{sum_dlac_imperfect_cp:.4e}"
    sum_slac_cp_sci = f"{sum_slac_cp:.4e}"

    # Create a dictionary with the sums
    consumer_payments = {
        'CEM': sum_cem_cp,
        'PF': sum_pf_cp,
        'DLAC-p': sum_dlac_cp,
        'DLAC-i': sum_dlac_imperfect_cp,
        'SLAC': sum_slac_cp
    }

    # # Sort the dictionary by values in descending order
    # sorted_consumer_payments = sorted(consumer_payments.items(), key=lambda item: item[1], reverse=True)

    consumer_payments_dic[case_name] = consumer_payments

    # consumer_payments_ratio_df = percent_change_df(consumer_payments)

    # consumer_payments_ratio_dic[case_name] = consumer_payments_ratio_df

    ###================================================================================================
    ### Calculate Total System Costs
    ###================================================================================================

    # nse_costs_dic[case_name] = {} # XXX why is this here?

    cem_total_system_costs = cem_system_costs.sum().sum() + cem_total_nse_cost
    pf_total_system_costs = pf_system_costs.sum().sum() + pf_sum_nse_cost
    dlac_total_system_costs = dlac_system_costs.sum().sum() + dlac_sum_nse_cost
    dlac_imperfect_total_system_costs = dlac_imperfect_system_costs.sum().sum() + dlac_imperfect_sum_nse_cost
    slac_total_system_costs = slac_system_costs.sum().sum() + slac_sum_nse_cost

    cem_total_system_costs_sci = f"{cem_total_system_costs:.4e}"
    pf_total_system_costs_sci = f"{pf_total_system_costs:.4e}"
    dlac_total_system_costs_sci = f"{dlac_total_system_costs:.4e}"
    dlac_imperfect_total_system_costs_sci = f"{dlac_imperfect_total_system_costs:.4e}"
    slac_total_system_costs_sci = f"{slac_total_system_costs:.4e}"

    total_inv_op_costs = {
        'CEM': cem_costs.sum().sum(),
        'PF': pf_costs.sum().sum(),
        'DLAC-p': dlac_costs.sum().sum(),
        'DLAC-i': dlac_imperfect_costs.sum().sum(),
        'SLAC': slac_costs.sum().sum()
    }

    total_inv_op_costs_dic[case_name] = total_inv_op_costs

    # Create a dictionary with the total costs
    total_system_costs = {
        'CEM': cem_total_system_costs,
        'PF': pf_total_system_costs,
        'DLAC-p': dlac_total_system_costs,
        'DLAC-i': dlac_imperfect_total_system_costs,
        'SLAC': slac_total_system_costs
    }

    # # Sort the dictionary by values in descending order
    # sorted_total_system_costs = sorted(total_system_costs.items(), key=lambda item: item[1], reverse=True)

    # # Print the sorted total costs
    # for case, cost in sorted_total_system_costs:
    #     print("case_name: " f"{case}: {cost:.4e}")

    total_system_costs_dic[case_name] = total_system_costs

    ###================================================================================================
    ### Calculate global objective total
    ###================================================================================================
    cem_objective_total = cem_total_system_costs + cem_sum_ur_cost
    pf_objective_total = pf_total_system_costs + pf_sum_ur_cost
    dlac_objective_total = dlac_total_system_costs + dlac_sum_ur_cost
    dlac_imperfect_objective_total = dlac_imperfect_total_system_costs + dlac_imperfect_sum_ur_cost
    slac_objective_total = slac_total_system_costs + slac_sum_ur_cost

    cem_objective_total_sci = f"{cem_objective_total:.4e}"
    pf_objective_total_sci = f"{pf_objective_total:.4e}"
    dlac_objective_total_sci = f"{dlac_objective_total:.4e}"
    dlac_imperfect_objective_total_sci = f"{dlac_imperfect_objective_total:.4e}"
    slac_objective_total_sci = f"{slac_objective_total:.4e}"

    # Create a dictionary with the total costs
    total_objective = {
        'CEM': cem_objective_total,
        'PF': pf_objective_total,
        'DLAC-p': dlac_objective_total,
        'DLAC-i': dlac_imperfect_objective_total,
        'SLAC': slac_objective_total
    }

    total_objective_dic[case_name] = total_objective

    ###================================================================================================
    ### Calculate Operating Costs
    ###================================================================================================
    operating_costs = ['Fixed_OM_cost_MW', 'Var_OM_cost_out', 'Fuel_cost', 'Var_OM_cost_in', 'StartCost']

    cem_operating_costs = cem_costs[operating_costs].sum().sum()
    pf_operating_costs = pf_costs[operating_costs].sum().sum()
    dlac_operating_costs = dlac_costs[operating_costs].sum().sum()
    dlac_imperfect_operating_costs = dlac_imperfect_costs[operating_costs].sum().sum()
    slac_operating_costs = slac_costs[operating_costs].sum().sum()

    cem_operating_costs_sci = f"{cem_operating_costs:.4e}"
    pf_operating_costs_sci = f"{pf_operating_costs:.4e}"
    dlac_operating_costs_sci = f"{dlac_operating_costs:.4e}"
    dlac_imperfect_operating_costs_sci = f"{dlac_imperfect_operating_costs:.4e}"
    slac_operating_costs_sci = f"{slac_operating_costs:.4e}"

    # Create a dictionary with the operating costs
    total_operating_costs = {
        'CEM': cem_operating_costs,
        'PF': pf_operating_costs,
        'DLAC-p': dlac_operating_costs,
        'DLAC-i': dlac_imperfect_operating_costs,
        'SLAC': slac_operating_costs
    }

    total_operating_costs_dic[case_name] = total_operating_costs

    ###================================================================================================
    ### Calculate Fixed OM Costs
    ###================================================================================================

    cem_fixed_om_costs = cem_costs['Fixed_OM_cost_MW'].sum().sum()
    pf_fixed_om_costs = pf_costs['Fixed_OM_cost_MW'].sum().sum()
    dlac_fixed_om_costs = dlac_costs['Fixed_OM_cost_MW'].sum().sum()
    dlac_imperfect_fixed_om_costs = dlac_imperfect_costs['Fixed_OM_cost_MW'].sum().sum()
    slac_fixed_om_costs = slac_costs['Fixed_OM_cost_MW'].sum().sum()

    cem_fixed_om_costs_sci = f"{cem_fixed_om_costs:.4e}"
    pf_fixed_om_costs_sci = f"{pf_fixed_om_costs:.4e}"
    dlac_fixed_om_costs_sci = f"{dlac_fixed_om_costs:.4e}"
    dlac_imperfect_fixed_om_costs_sci = f"{dlac_imperfect_fixed_om_costs:.4e}"
    slac_fixed_om_costs_sci = f"{slac_fixed_om_costs:.4e}"

    # Create a dictionary with the fixed OM costs
    total_fixed_om_costs = {
        'CEM': cem_fixed_om_costs,
        'PF': pf_fixed_om_costs,
        'DLAC-p': dlac_fixed_om_costs,
        'DLAC-i': dlac_imperfect_fixed_om_costs,
        'SLAC': slac_fixed_om_costs
    }

    total_fixed_om_costs_dic[case_name] = total_fixed_om_costs

    ###================================================================================================
    ### Calculate Variable OM Costs
    ###================================================================================================

    cem_var_om_costs = cem_costs['Var_OM_cost_out'].sum().sum()
    pf_var_om_costs = pf_costs['Var_OM_cost_out'].sum().sum()
    dlac_var_om_costs = dlac_costs['Var_OM_cost_out'].sum().sum()
    dlac_imperfect_var_om_costs = dlac_imperfect_costs['Var_OM_cost_out'].sum().sum()
    slac_var_om_costs = slac_costs['Var_OM_cost_out'].sum().sum()

    cem_var_om_costs_sci = f"{cem_var_om_costs:.4e}"
    pf_var_om_costs_sci = f"{pf_var_om_costs:.4e}"
    dlac_var_om_costs_sci = f"{dlac_var_om_costs:.4e}"
    dlac_imperfect_var_om_costs_sci = f"{dlac_imperfect_var_om_costs:.4e}"
    slac_var_om_costs_sci = f"{slac_var_om_costs:.4e}"

    # Create a dictionary with the variable OM costs
    total_var_om_costs = {
        'CEM': cem_var_om_costs,
        'PF': pf_var_om_costs,
        'DLAC-p': dlac_var_om_costs,
        'DLAC-i': dlac_imperfect_var_om_costs,
        'SLAC': slac_var_om_costs
    }

    total_var_om_out_costs_dic[case_name] = total_var_om_costs

    ###================================================================================================
    ### Calculate Fuel Costs
    ###================================================================================================

    cem_fuel_costs = cem_costs['Fuel_cost'].sum().sum()
    pf_fuel_costs = pf_costs['Fuel_cost'].sum().sum()
    dlac_fuel_costs = dlac_costs['Fuel_cost'].sum().sum()
    dlac_imperfect_fuel_costs = dlac_imperfect_costs['Fuel_cost'].sum().sum()
    slac_fuel_costs = slac_costs['Fuel_cost'].sum().sum()

    cem_fuel_costs_sci = f"{cem_fuel_costs:.4e}"
    pf_fuel_costs_sci = f"{pf_fuel_costs:.4e}"
    dlac_fuel_costs_sci = f"{dlac_fuel_costs:.4e}"
    dlac_imperfect_fuel_costs_sci = f"{dlac_imperfect_fuel_costs:.4e}"
    slac_fuel_costs_sci = f"{slac_fuel_costs:.4e}"

    # Create a dictionary with the fuel costs
    total_fuel_costs = {
        'CEM': cem_fuel_costs,
        'PF': pf_fuel_costs,
        'DLAC-p': dlac_fuel_costs,
        'DLAC-i': dlac_imperfect_fuel_costs,
        'SLAC': slac_fuel_costs
    }

    total_fuel_costs_dic[case_name] = total_fuel_costs

    ###================================================================================================
    ### Calculate Variable OM Costs
    ###================================================================================================

    cem_var_om_in_costs = cem_costs['Var_OM_cost_in'].sum().sum()
    pf_var_om_in_costs = pf_costs['Var_OM_cost_in'].sum().sum()
    dlac_var_om_in_costs = dlac_costs['Var_OM_cost_in'].sum().sum()
    dlac_imperfect_var_om_in_costs = dlac_imperfect_costs['Var_OM_cost_in'].sum().sum()
    slac_var_om_in_costs = slac_costs['Var_OM_cost_in'].sum().sum()

    cem_var_om_in_costs_sci = f"{cem_var_om_in_costs:.4e}"
    pf_var_om_in_costs_sci = f"{pf_var_om_in_costs:.4e}"
    dlac_var_om_in_costs_sci = f"{dlac_var_om_in_costs:.4e}"
    dlac_imperfect_var_om_in_costs_sci = f"{dlac_imperfect_var_om_in_costs:.4e}"
    slac_var_om_in_costs_sci = f"{slac_var_om_in_costs:.4e}"

    # Create a dictionary with the variable OM costs
    total_var_om_in_costs = {
        'CEM': cem_var_om_in_costs,
        'PF': pf_var_om_in_costs,
        'DLAC-p': dlac_var_om_in_costs,
        'DLAC-i': dlac_imperfect_var_om_in_costs,
        'SLAC': slac_var_om_in_costs
    }

    total_var_om_in_costs_dic[case_name] = total_var_om_in_costs

    ###================================================================================================
    ### Calculate Start Costs
    ###================================================================================================

    cem_start_costs = cem_costs['StartCost'].sum().sum()
    pf_start_costs = pf_costs['StartCost'].sum().sum()
    dlac_start_costs = dlac_costs['StartCost'].sum().sum()
    dlac_imperfect_start_costs = dlac_imperfect_costs['StartCost'].sum().sum()
    slac_start_costs = slac_costs['StartCost'].sum().sum()

    cem_start_costs_sci = f"{cem_start_costs:.4e}"
    pf_start_costs_sci = f"{pf_start_costs:.4e}"
    dlac_start_costs_sci = f"{dlac_start_costs:.4e}"
    dlac_imperfect_start_costs_sci = f"{dlac_imperfect_start_costs:.4e}"
    slac_start_costs_sci = f"{slac_start_costs:.4e}"

    # Create a dictionary with the start costs
    total_start_costs = {
        'CEM': cem_start_costs,
        'PF': pf_start_costs,
        'DLAC-p': dlac_start_costs,
        'DLAC-i': dlac_imperfect_start_costs,
        'SLAC': slac_start_costs
    }

    total_start_costs_dic[case_name] = total_start_costs

    ###================================================================================================
    ### Calculate Charge Costs
    ###================================================================================================

    cem_charge_costs = cem_raw_upd['Charge_cost'].sum().sum()
    pf_charge_costs = pf_raw['Charge_cost'].sum().sum()
    dlac_charge_costs = dlac_raw['Charge_cost'].sum().sum()
    dlac_imperfect_charge_costs = dlac_imperfect_raw['Charge_cost'].sum().sum()
    slac_charge_costs = slac_raw['Charge_cost'].sum().sum()

    cem_charge_costs_sci = f"{cem_charge_costs:.4e}"
    pf_charge_costs_sci = f"{pf_charge_costs:.4e}"
    dlac_charge_costs_sci = f"{dlac_charge_costs:.4e}"
    dlac_imperfect_charge_costs_sci = f"{dlac_imperfect_charge_costs:.4e}"
    slac_charge_costs_sci = f"{slac_charge_costs:.4e}"

    # Create a dictionary with the charge costs
    total_charge_costs = {
        'CEM': cem_charge_costs,
        'PF': pf_charge_costs,
        'DLAC-p': dlac_charge_costs,
        'DLAC-i': dlac_imperfect_charge_costs,
        'SLAC': slac_charge_costs
    }

    total_charge_costs_dic[case_name] = total_charge_costs

    ###================================================================================================
    ### Calculate Profit Margin Rate
    ###================================================================================================

    # dfGen.index = gen_print_names
    char_str = ['Inv_Cost_per_MWyr', 'Inv_Cost_per_MWhyr', 'Fixed_OM_Cost_per_MWyr']
    dfGen_annual_costs = dfGen[char_str]

    # if the Inv_Cost_per_MWhyr in each row is > 0, then multiply the Inv_Cost_per_MWhyr by the Max_Duration
    inv_cost_per_mwhyr2mwyr = dfGen.apply(
        lambda row: row['Inv_Cost_per_MWhyr'] * row['Existing_Cap_MWh'] / row['Existing_Cap_MW'] \
            if row['Inv_Cost_per_MWhyr'] > 0 else row['Inv_Cost_per_MWhyr'],
        axis=1
    )

    # Replace 'Inv_Cost_per_MWhyr' in dfGen_annual_costs with inv_cost_permwhyr2mwyr and title it 'Inv_Cost_per_Energy_by_hour'
    dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
    dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)

    dfGen_annual_costs_sum = dfGen_annual_costs.sum(axis=1) # $/MWyr

    # \alpha_{i,\Pi} = \frac{P_{i,\Pi}}{C_{i,\Pi}}
    cem_annual_profit2costs = cem_unit_profit_mw / dfGen_annual_costs_sum
    pf_annual_profit2costs = pf_unit_profit_mw / dfGen_annual_costs_sum
    dlac_annual_profit2costs = dlac_unit_profit_mw / dfGen_annual_costs_sum
    dlac_imperfect_annual_profit2costs = dlac_imperfect_unit_profit_mw / dfGen_annual_costs_sum
    slac_annual_profit2costs = slac_unit_profit_mw / dfGen_annual_costs_sum

    cem_ap2c_perc = cem_annual_profit2costs * 100
    pf_ap2c_perc = pf_annual_profit2costs * 100
    dlac_ap2c_perc = dlac_annual_profit2costs * 100
    dlac_imperfect_ap2c_perc = dlac_imperfect_annual_profit2costs * 100
    slac_ap2c_perc = slac_annual_profit2costs * 100

    cem_ap2c_perc_sci = cem_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    pf_ap2c_perc_sci = pf_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_ap2c_perc_sci = dlac_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_imperfect_perc_ap2c_sci = dlac_imperfect_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    slac_ap2c_perc_sci = slac_ap2c_perc.apply(lambda x: f"{x:.2f}%")


    percentage_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_ap2c_perc,
        'PF': pf_ap2c_perc,
        'DLAC-p': dlac_ap2c_perc,
        'DLAC-i': dlac_imperfect_ap2c_perc,
        'SLAC': slac_ap2c_perc
    })

    percentage_profits_dic[case_name] = percentage_profit_df
    ### DONE


Case Name: Thermal_Base



C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:651: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:652: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)


Case Name: 2_Hr_BESS



C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:651: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:652: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)


Case Name: 2_Hr_BESS_Fuelx2



...

C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:651: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:652: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)


Case Name: 10_Hr_BESS



C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:651: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:652: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)


Case Name: 10_Hr_BESS_Fuelx2



C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:651: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
C:\Users\ks885\AppData\Local\Temp\ipykernel_24624\1307071175.py:652: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)


In [11]:
# initialize a dictinoary to store the results
results_dic = {}

In [12]:
def format_percentage(val):
    try:
        fval = float(val)
        if fval == 0:
            return "0"
        else:
            return f"{fval:.2f}"
    except Exception:
        return val

In [13]:
def print_percent_change_csv_latex(df, filename, model_norm):
    change_df = percent_change_df(df, model_norm)

    # print to csv
    change_df.to_csv(csv_path + filename + '.csv', index=False)
    # update for printing to latex
    change_df_formatted = change_df.copy()
    # Replace underscores with spaces in 'Case Name' column if it exists
    if 'Case Name' in change_df_formatted.columns:
        change_df_formatted['Case Name'] = change_df_formatted['Case Name'].str.replace('_', ' ')
    for col in change_df_formatted.columns:
        if col != 'Case Name':
            change_df_formatted[col] = change_df_formatted[col].apply(format_percentage)

    # Write LaTeX table manually so that \% only goes on numeric values
    with open(latex_path + filename + '.tex', 'w') as f:
        f.write("\\begin{tabular}{l" + "c" * (len(change_df_formatted.columns) - 1) + "}\n")
        f.write("\\toprule\n")
        f.write(" & ".join(change_df_formatted.columns) + " \\\\\n")
        f.write("\\midrule\n")
        for _, row in change_df_formatted.iterrows():
            row_strs = []
            for col, val in row.items():
                if col == 'Case Name':
                    row_strs.append(str(val))
                else:
                    try:
                        fval = float(val)
                        # Only add \% if val is a number and not nan
                        if not pd.isnull(fval):
                            row_strs.append(f"{val}\\%")
                        else:
                            row_strs.append(str(val))
                    except Exception:
                        row_strs.append(str(val))
            f.write(" & ".join(row_strs) + " \\\\\n")
        f.write("\\bottomrule\n")
        f.write("\\end{tabular}\n")

    return change_df_formatted

In [14]:
# create dataframe from total_nse_dic with rows for each case_name in case_names
total_nse_df = pd.DataFrame.from_dict(total_nse_dic, orient='index').reset_index()
total_nse_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
# replace strings in the 'Case Name' column
total_nse_df['Case Name'] = total_nse_df['Case Name'].str.replace('_', ' ')
total_nse_df.to_csv(csv_path + 'total_nse.csv', index=False)
total_nse_change_df = print_percent_change_csv_latex(total_nse_df, 'total_nse_change', 'PF')

In [15]:
total_unmet_rsv_df = pd.DataFrame.from_dict(total_unmet_rsv_dic, orient='index').reset_index()
total_unmet_rsv_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
# replace strings in the 'Case Name' column
total_unmet_rsv_df['Case Name'] = total_unmet_rsv_df['Case Name'].str.replace('_', ' ')
total_unmet_rsv_df.to_csv(csv_path + 'total_unmet_rsv.csv', index=False)
total_unmet_rsv_change_df =print_percent_change_csv_latex(total_unmet_rsv_df, 'total_unmet_rsv_change', 'PF')

In [16]:
# Create a dataframe from consumer_payments_dic with rows for each case_name in case_names
consumer_payments_df = pd.DataFrame.from_dict(consumer_payments_dic, orient='index').reset_index()
consumer_payments_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
# replace strings in the 'Case Name' column
consumer_payments_df['Case Name'] = consumer_payments_df['Case Name'].str.replace('_', ' ')
consumer_payments_df.to_csv(csv_path + 'consumer_payments.csv', index=False)
consumer_payments_change_df = print_percent_change_csv_latex(consumer_payments_df, 'consumer_payments_change', 'CEM')

In [17]:
total_inv_costs_df = pd.DataFrame.from_dict(total_investment_costs_dic, orient='index').reset_index()
total_inv_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
# replace strings in the 'Case Name' column
total_inv_costs_df['Case Name'] = total_inv_costs_df['Case Name'].str.replace('_', ' ')
total_inv_costs_df.to_csv(csv_path + 'total_inv_costs.csv', index=False)
total_inv_costs_change_df = print_percent_change_csv_latex(total_inv_costs_df, 'total_inv_costs_change', 'PF')

In [18]:
total_inv_op_costs_df = pd.DataFrame.from_dict(total_inv_op_costs_dic, orient='index').reset_index()
total_inv_op_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_inv_op_costs_df.to_csv(csv_path + 'total_inv_op_costs.csv', index=False)
total_inv_op_costs_change_df = print_percent_change_csv_latex(total_inv_op_costs_df, 'total_inv_op_costs_change', 'PF')

In [19]:
# Create a dataframe from total_system_costs_dic with rows for each case_name in case_names
total_operating_costs_df = pd.DataFrame.from_dict(total_operating_costs_dic, orient='index').reset_index()
total_operating_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_operating_costs_df.to_csv(csv_path + 'total_operational_costs.csv', index=False)
total_operating_costs_change_df = print_percent_change_csv_latex(total_operating_costs_df, 'total_operating_costs_change', 'PF')

In [20]:
total_fixed_om_costs_df = pd.DataFrame.from_dict(total_fixed_om_costs_dic, orient='index').reset_index()
total_fixed_om_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_fixed_om_costs_df.to_csv(csv_path + 'total_fixed_om_costs.csv', index=False)
total_fixed_om_costs_change_df = print_percent_change_csv_latex(total_fixed_om_costs_df, 'total_fixed_om_costs_change', 'PF')

In [21]:
total_var_om_out_costs_df = pd.DataFrame.from_dict(total_var_om_out_costs_dic, orient='index').reset_index()
total_var_om_out_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_var_om_out_costs_df.to_csv(csv_path + 'total_var_om_out_costs.csv', index=False)
total_var_om_out_costs_change_df = print_percent_change_csv_latex(total_var_om_out_costs_df, 'total_var_om_out_costs_change', 'PF')

In [22]:
total_fuel_costs_df = pd.DataFrame.from_dict(total_fuel_costs_dic, orient='index').reset_index()
total_fuel_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_fuel_costs_df.to_csv(csv_path + 'total_fuel_costs.csv', index=False)
total_fuel_costs_change_df = print_percent_change_csv_latex(total_fuel_costs_df, 'total_fuel_costs_change', 'PF')

In [23]:
total_var_om_in_costs_df = pd.DataFrame.from_dict(total_var_om_in_costs_dic, orient='index').reset_index()
total_var_om_in_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_var_om_in_costs_df.to_csv(csv_path + 'total_var_om_in_costs.csv', index=False)
total_var_om_in_costs_change_df = print_percent_change_csv_latex(total_var_om_in_costs_df, 'total_var_om_in_costs_change', 'PF')

In [24]:
total_start_costs_df = pd.DataFrame.from_dict(total_start_costs_dic, orient='index').reset_index()
total_start_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_start_costs_df.to_csv(csv_path + 'total_start_costs.csv', index=False)
total_start_costs_change_df = print_percent_change_csv_latex(total_start_costs_df, 'total_start_costs_change', 'PF')

In [25]:
total_charge_costs_df = pd.DataFrame.from_dict(total_charge_costs_dic, orient='index').reset_index()
total_charge_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_charge_costs_df.to_csv(csv_path + 'total_charge_costs.csv', index=False)
total_charge_costs_change_df = print_percent_change_csv_latex(total_charge_costs_df, 'total_charge_costs_change', 'PF')

In [26]:
total_unmet_rsv_cost_df = pd.DataFrame.from_dict(total_unmet_rsv_cost_dic, orient='index').reset_index()
total_unmet_rsv_cost_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_unmet_rsv_cost_df.to_csv(csv_path + 'total_unmet_rsv_costs.csv', index=False)
total_unmet_rsv_costs_change_df = print_percent_change_csv_latex(total_unmet_rsv_cost_df, 'total_unmet_rsv_costs_change', 'PF')

In [27]:
total_nse_cost_df = pd.DataFrame.from_dict(total_nse_cost_dic, orient='index').reset_index()
total_nse_cost_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_nse_cost_df.to_csv(csv_path + 'total_nse_costs.csv', index=False)
total_nse_costs_change_df = print_percent_change_csv_latex(total_nse_cost_df, 'total_nse_costs_change', 'PF')

In [28]:
# Create a dataframe from total_system_costs_dic with rows for each case_name in case_names
total_system_costs_df = pd.DataFrame.from_dict(total_system_costs_dic, orient='index').reset_index()
total_system_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_system_costs_df.to_csv(csv_path + 'total_system_costs.csv', index=False)
total_system_costs_change_df = print_percent_change_csv_latex(total_system_costs_df, 'total_system_costs_change', 'PF')

In [29]:
total_objective_df = pd.DataFrame.from_dict(total_objective_dic, orient='index').reset_index()
total_objective_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
total_objective_df.to_csv(csv_path + 'total_objective.csv', index=False)
total_objective_change_df = print_percent_change_csv_latex(total_objective_df, 'tableX_total_objective_change_wrt_PF', 'PF')

In [30]:
# save all dataframes to pdf
total_change_fln = 'total_change_dataframes_' + date + '.pdf'

# Create a PDF file
with PdfPages(pdf_path + total_change_fln) as pdf:
    # List of total_change dataframes
    total_change_dataframes = [
        total_charge_costs_change_df,
        total_fixed_om_costs_change_df,
        total_fuel_costs_change_df,
        total_inv_costs_change_df,
        total_inv_op_costs_change_df,
        total_start_costs_change_df,
        total_var_om_in_costs_change_df,
        total_var_om_out_costs_change_df,
        total_system_costs_change_df,
        total_unmet_rsv_costs_change_df,
        total_nse_costs_change_df,
        total_objective_change_df
    ]

    # Titles for each dataframe
    titles = [
        'Total Change in Charge Costs',
        'Total Change in Fixed OM Costs',
        'Total Change in Fuel Costs',
        'Total Change in Investment Costs',
        'Total Change in Investment and Operating Costs',
        'Total Change in Start Costs',
        'Total Change in Variable OM In Costs',
        'Total Change in Variable OM Out Costs',
        'Total Change in System Costs',
        'Total Change in Unmet Reserves Costs',
        'Total Change in NSE Costs',
        'Total Change in Objective Costs'
    ]

    # Iterate over dataframes and save each to the PDF
    for df, title in zip(total_change_dataframes, titles):
        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(title, fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)

### These nse costs are calculated exogenously

In [31]:

# multiply the nse_totals by 5000 to get the total nse costs and compare against the total_nse_cost_df
nse_cost_from_totals = total_nse_df.copy()
for col in total_nse_cost_df.columns:
    if col != 'Case Name':
        nse_cost_from_totals[col] = nse_cost_from_totals[col].astype(float) * 5000

# Convert the values to scientific notation
for col in nse_cost_from_totals.columns:
    if col != 'Case Name':
        nse_cost_from_totals[col] = nse_cost_from_totals[col].apply(lambda x: f"{x:.4e}")

nse_cost_from_totals

,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal Base,5.8765e+07,5.8765e+07,5.8765e+07,6.9045e+07,5.8765e+07
1,2 Hr BESS,1.0797e+08,1.0800e+08,1.0800e+08,1.2728e+08,1.4436e+08
2,2 Hr BESS Fuelx2,6.2225e+07,6.2345e+07,6.2345e+07,1.6892e+08,1.5109e+08
3,4 Hr BESS,6.5465e+07,6.5545e+07,6.5545e+07,1.5180e+08,1.2546e+08
4,4 Hr BESS Fuelx2,0.0000e+00,1.2528e+05,1.2528e+05,8.3135e+07,6.6935e+07
5,4 Hr BESS Fuelx3,0.0000e+00,1.8146e+05,1.8146e+05,5.8345e+07,2.9373e+07
6,4 Hr BESS Fuelx4,8.2725e+07,8.3525e+07,8.3525e+07,1.1199e+08,1.0494e+08
7,6 Hr BESS,3.4244e+07,3.5166e+07,3.5166e+07,1.2730e+08,1.0938e+08
8,6 Hr BESS Fuelx2,0.0000e+00,1.1618e+05,1.1618e+05,7.6235e+07,6.1095e+07
9,8 Hr BESS,1.7216e+07,1.7260e+07,1.7260e+07,1.0744e+08,8.3325e+07


In [32]:
from matplotlib.backends.backend_pdf import PdfPages

# BEGIN: Add unit profit tables to PDF

# create a date stamp of just the year, month and day
date = datetime.datetime.now()
date = date.strftime("%Y-%m-%d-%H-%M")

In [33]:
unit_profit_dic

{'Thermal_Base':                                    Resource           CEM           PF  \
 0        NG 2-on-1 Combined Cycle (F-Frame) -1.626201e-11  3192.831171   
 1           NG Combustion Turbine (F-Frame)  3.151297e-12  2488.898774   
 2  Land-Based Wind - Class 1 - Technology 1  2.472086e-10 -1954.105949   
 3                      Utility PV - Class 1  1.326542e-11   -47.715759   
 
         DLAC-p        DLAC-i          SLAC  
 0 -9694.129492  18841.384728 -26631.407421  
 1  1410.944878  18786.988608   -519.799157  
 2 -6006.680429  -2103.636802 -33798.896676  
 3 -3328.018208   2474.690266  -9751.959243  ,
 '2_Hr_BESS':                                    Resource           CEM           PF  \
 0        NG 2-on-1 Combined Cycle (F-Frame) -1.070732e-09  2436.282296   
 1           NG Combustion Turbine (F-Frame)  1.543888e-11   590.078850   
 2  Land-Based Wind - Class 1 - Technology 1  2.290413e-10 -1850.534518   
 3                      Utility PV - Class 1 -2.390585e-10  204

In [34]:
# create a dataframe that appends each unit profit dataframe in unit_profit_dic
# also update the resource names to get_printable_resources_names
# create a dataframe that appends each unit profit dataframe in unit_profit_dic
# also update the resource names to get_printable_resource_names
all_unit_profits_df = pd.concat(
    [
        df.assign(
            Case=case,
            Resource=df['Resource'].apply(get_printable_resource_names)
        )
        for case, df in unit_profit_dic.items()
    ],
    ignore_index=True
)

In [35]:
all_unit_profits_df

,Resource,CEM,PF,DLAC-p,DLAC-i,SLAC,Case
0,NG CC,-1.626201e-11,3192.831171,-9694.129492,18841.384728,-26631.407421,Thermal_Base
1,NG CT,3.151297e-12,2488.898774,1410.944878,18786.988608,-519.799157,Thermal_Base
2,Wind,2.472086e-10,-1954.105949,-6006.680429,-2103.636802,-33798.896676,Thermal_Base
3,Solar,1.326542e-11,-47.715759,-3328.018208,2474.690266,-9751.959243,Thermal_Base
4,NG CC,-1.070732e-09,2436.282296,-7530.824112,-21939.105786,-29142.507982,2_Hr_BESS
...,...,...,...,...,...,...,...
59,NG CC,4.532602e-10,10182.428018,7454.401546,-49664.240720,-48229.091432,10_Hr_BESS_Fuelx2
60,NG CT,-2.397021e-11,10342.920392,9726.202753,-55802.030366,-50550.106961,10_Hr_BESS_Fuelx2
61,Wind,2.810783e-10,2101.751496,40.133814,-3306.982227,-7673.293035,10_Hr_BESS_Fuelx2
62,Solar,-2.394236e-10,-326.214634,2105.092016,3538.987117,-2547.640382,10_Hr_BESS_Fuelx2


In [36]:
unit_profits_fln =  'unit_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(pdf_path + unit_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        unit_profit_df = unit_profit_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=unit_profit_df.values, colLabels=unit_profit_df.columns, cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'Unit Profits for {case_name} ($/MW)', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [61]:
all_perc_profits = pd.concat(
    [
        df.assign(
            Case=case,
            Resource=df['Resource'].apply(get_printable_resource_names)
        )
        for case, df in percentage_profits_dic.items()
    ],
    ignore_index=True
)

# create a copy of all_perc_profits to modify
all_perc_profits_rounded = all_perc_profits.copy()
# rewrite all elements in model columns as strings and add a % sign
# Rewrite all elements in model columns as strings with no decimal points and add a % sign
model_cols = ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
for col in model_cols:
    if col in all_perc_profits_rounded.columns:
        all_perc_profits_rounded[col] = all_perc_profits_rounded[col].apply(
            lambda x: f"{int(round(x))}%" if isinstance(x, (int, float)) else x
        )
# save all percentage profits to csv
all_perc_profits.to_csv(csv_path + 'Table3etc_percentage_profits.csv', index=False)

In [38]:
percentage_profits_fln =  'table_percentage_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(pdf_path + percentage_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        percentage_profits_df = percentage_profits_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=percentage_profits_df.values, \
                         colLabels=percentage_profits_df.columns, \
                            cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'% Profits over Fixed Costs for {case_name}', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [59]:
bess_case_names = [name for name in case_names if name != 'Thermal_Base']

In [ ]:
# loop through cases, compute average cpr across models,
# and calculate two average pmr's: across models and generators,

# initialize all cases average dataframe with rows as generators and columns as models
all_cases_avg_df = pd.DataFrame


for case_name in percentage_profits_dic:


{'Thermal_Base':                                    Resource           CEM        PF  \
 0        NG 2-on-1 Combined Cycle (F-Frame) -1.980829e-14  3.889097   
 1           NG Combustion Turbine (F-Frame)  4.491871e-15  3.547685   
 2  Land-Based Wind - Class 1 - Technology 1  2.136037e-13 -1.688470   
 3                      Utility PV - Class 1  1.355000e-14 -0.048739   
 
       DLAC-p     DLAC-i       SLAC  
 0 -11.808143  22.950153 -32.438958  
 1   2.011166  26.779042  -0.740924  
 2  -5.190147  -1.817674 -29.204357  
 3  -3.399414   2.527780  -9.961167  ,
 '2_Hr_BESS':                                    Resource           CEM         PF  \
 0        NG 2-on-1 Combined Cycle (F-Frame) -6.521138e-13   1.483783   
 1           NG Combustion Turbine (F-Frame)  1.100332e-14   0.420550   
 2  Land-Based Wind - Class 1 - Technology 1  1.979060e-13  -1.598977   
 3                      Utility PV - Class 1 -2.441870e-13   2.090002   
 4       Utility-Scale Battery Storage - 2Hr -1.48461

In [40]:
# Function to create LaTeX tables from a dictionary and save them to the same file
def create_latex_tables_from_dict(data_dict, output_path, title, float_format=None):
    """
    Create a single LaTeX file containing tables for each DataFrame in a dictionary.

    Args:
        data_dict (dict): A dictionary where keys are table names and values are DataFrames.
        output_path (str): The path to save the LaTeX file.
        title (str): The base filename for the LaTeX file.
        float_format (str or None): Format string for floats, e.g., '%.2f' for 2 decimal points.

    Returns:
        None
    """
    with open(output_path + title + '.tex', 'w') as latex_file:
        # Write the LaTeX document header
        latex_file.write("\\documentclass{article}\n")
        latex_file.write("\\usepackage{booktabs}\n")
        latex_file.write("\\usepackage{longtable}\n")
        latex_file.write("\\begin{document}\n")

        # Loop through the dictionary and add each DataFrame as a table
        for table_name, df in data_dict.items():
            latex_file.write("\\hline\n")
            latex_file.write(f"\\multicolumn{{{len(df.columns)}}}{{|c|}}{{{table_name}}} \\\\\n")
            latex_file.write("\\hline\n")
            if float_format is not None:
                latex_file.write(df.to_latex(index=False, escape=False, longtable=True, float_format=float_format))
            else:
                latex_file.write(df.to_latex(index=False, escape=False, longtable=True))
            latex_file.write("\n\n")

        # Write the LaTeX document footer
        latex_file.write("\\end{document}\n")

In [ ]:
def create_pmr_cpr_latex_table_all_cases(percentage_profits_dic, consumer_payments_change_df, output_path, filename, models=None):
    """
    Create a LaTeX file with PMR and CPR tables for all cases.

    Args:
        percentage_profits_dic (dict): Dictionary of DataFrames with PMR values for each case.
        consumer_payments_change_df (pd.DataFrame): DataFrame with CPR values for all cases.
        output_path (str): Path to save the LaTeX file.
        filename (str): Filename for the LaTeX file (without extension).
    """

    with open(output_path + filename + '.tex', 'w') as f:

        for case_name in percentage_profits_dic:
            percentage_profits_df = percentage_profits_dic[case_name]
            # Find the row in consumer_payments_change_df for this case
            cpr_row = consumer_payments_change_df[consumer_payments_change_df['Case Name'].str.replace('_', ' ') == case_name.replace('_', ' ')]
            if not cpr_row.empty:
                cpr_row = cpr_row.iloc[0][models].values
            else:
                cpr_row = [''] * len(models)

            pmr_rows = percentage_profits_df['Resource'].tolist()
            pmr_data = percentage_profits_df[models].values

            f.write("\\begin{table}[ht]\n")
            f.write("\\centering\n")
            f.write(f"\\caption{{Profit Margin Rates (PMR) and Consumer Payments Ratio (CPR) for {case_name.replace('_', ' ')}.}}\n")
            f.write("\\begin{tabular}{l" + "c" * len(models) + "}\n") # number of c's equal to the number of PMR columns
            f.write("\\toprule\n")
            for col in models:
                f.write(f"& \\multicolumn{{1}}{{l}}{{\\textbf{{{col}}}}} \n")
            f.write(" \\\\ \\hline\n")
            f.write("\\midrule\n")
            f.write("\\textit{PMR} & & & & \\\\\n")
            for i, resource in enumerate(pmr_rows):

                printable_res_name = get_printable_resource_names(resource)
                # Round PMR values to integer and add % sign
                pmr_values = []
                for j in range(len(models)):
                    val = pmr_data[i, j]
                    if isinstance(val, (float, int)):
                        pmr_values.append(f"{int(round(val))}\\%")
                    else:
                        pmr_values.append(str(val))
                f.write(f"\\quad {printable_res_name} & " + " & ".join(pmr_values) + " \\\\\n")
            f.write("& & & & \\\\\n")
            # Format CPR values as percentages with a trailing \%
            formatted_cpr_row = [str(x) + r"\%" if isinstance(x, str) and not x.endswith(r"\%") else str(x) for x in cpr_row]
            f.write("\\textit{CPR} & " + " & ".join(int(round(formatted_cpr_row))) + " \\\\\n")
            f.write("\\bottomrule\n")
            f.write("\\end{tabular}\n")
            f.write(f"\\label{{table:{case_name}_pmr}}\n")
            f.write("\\end{table}\n\n")

        f.write("\\end{document}\n")

In [42]:
def create_selected_cases_pmr_cpr_latex_table(
    percentage_profits_dic, 
    consumer_payments_change_df, 
    output_path, 
    filename, 
    selected_cases, 
    models=None, 
    resource_print_names=None
):
    """
    Create a LaTeX table for PMR and CPR for a selected set of cases.

    Args:
        percentage_profits_dic (dict): Dictionary of DataFrames with PMR values for each case.
        consumer_payments_change_df (pd.DataFrame): DataFrame with CPR values for all cases.
        output_path (str): Path to save the LaTeX file.
        filename (str): Filename for the LaTeX file (without extension).
        selected_cases (list): List of case keys to include.
        models (list): List of model columns to include.
        resource_print_names (dict): Optional mapping from resource name to printable name.
    """
    if models is None:
        models = ['PF', 'DLAC-p', 'DLAC-i', 'SLAC']
    if resource_print_names is None:
        resource_print_names = {
            'NG 2-on-1 Combined Cycle (F-Frame)': 'NG CC',
            'NG Combustion Turbine (F-Frame)': 'NG CT',
            'Land-Based Wind - Class 1 - Technology 1': 'Wind',
            'Utility PV - Class 1': 'Solar',
            'Utility-Scale Battery Storage - 10Hr': 'BESS',
            'Utility-Scale Battery Storage - 2Hr': 'BESS',
            'Utility-Scale Battery Storage - 4Hr': 'BESS',
            'Utility-Scale Battery Storage - 6Hr': 'BESS',
            'Utility-Scale Battery Storage - 8Hr': 'BESS',
            'Utility-Scale Battery Storage - 10Hr': 'BESS',
        }

    with open(output_path + filename + '.tex', 'w') as f:
        f.write("\\begin{table}[]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{l" + "c" * len(models) + "}\n")
        f.write("\\hline\n")
        for i, model in enumerate(models):
            if i == 0:
                f.write("& \\multicolumn{1}{l}{\\textbf{" + model + "}} ")
            else:
                f.write("& \\multicolumn{1}{l}{\\textbf{" + model + "}} ")
        f.write("\\\\ \\hline\n")

        for case_key in selected_cases:
            # Print case title
            title = case_key.replace('_', ' ').replace('Hr', 'Hr').replace('Fuelx', ', Fuel Costs x')
            f.write(f"\\textbf{{{title}}} & " + " & " * (len(models)-1) + "\\\\\n")
            f.write(" \\quad \\textit{PMR}  & " + " & " * (len(models)-1) + "\\\\\n")

            pmr_df = percentage_profits_dic[case_key]
            # Get CPR row for this case
            cpr_row = consumer_payments_change_df[consumer_payments_change_df['Case Name'].str.replace('_', ' ') == case_key.replace('_', ' ')]
            if not cpr_row.empty:
                cpr_row = cpr_row.iloc[0][models].values
            else:
                cpr_row = [''] * len(models)

            # Print each resource row
            for _, row in pmr_df.iterrows():
                res_name = row['Resource']
                printable_res = resource_print_names.get(res_name, res_name)
                pmr_vals = []
                for model in models:
                    val = row[model]
                    try:
                        val = int(round(float(val)))
                        pmr_vals.append(f"{val}\\%")
                    except Exception:
                        pmr_vals.append(str(val))
                f.write(f"\\qquad {printable_res} & " + " & ".join(pmr_vals) + " \\\\\n")
            f.write("& " + " & " * (len(models)-1) + "\\\\\n")
            # CPR row
            formatted_cpr_row = []
            for x in cpr_row:
                try:
                    if isinstance(x, str) and x.endswith(r"\%"):
                        formatted_cpr_row.append(x)
                    else:
                        formatted_cpr_row.append(f"{float(x):.2f}\\%")
                except Exception:
                    formatted_cpr_row.append(str(x))
            f.write("\\quad \\textit{CPR} & " + " & ".join(formatted_cpr_row) + " \\\\\\hline\n")

        f.write("\\end{tabular}\n")
        f.write("\\end{table}\n")

In [43]:
create_selected_cases_pmr_cpr_latex_table(
    percentage_profits_dic=percentage_profits_dic,
    consumer_payments_change_df=consumer_payments_change_df,
    output_path=latex_path,
    filename='table5_pmr_cpr_4_Hr_BESS_cases',
    selected_cases=['4_Hr_BESS', '4_Hr_BESS_Fuelx2', '4_Hr_BESS_Fuelx3', '4_Hr_BESS_Fuelx4'],
    models= ['PF', 'DLAC-p', 'DLAC-i', 'SLAC']
)

In [44]:
create_selected_cases_pmr_cpr_latex_table(
    percentage_profits_dic=percentage_profits_dic,
    consumer_payments_change_df=consumer_payments_change_df,
    output_path=latex_path,
    filename='table7_pmr_cpr_4_Hr_BESS_cases',
    selected_cases=['2_Hr_BESS', '4_Hr_BESS', '6_Hr_BESS', '8_Hr_BESS', '10_Hr_BESS'],
    models= ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
)

In [45]:
create_selected_cases_pmr_cpr_latex_table(
    percentage_profits_dic=percentage_profits_dic,
    consumer_payments_change_df=consumer_payments_change_df,
    output_path=latex_path,
    filename='table8_pmr_cpr_4_Hr_BESS_cases',
    selected_cases=['2_Hr_BESS_Fuelx2', '4_Hr_BESS_Fuelx2', '6_Hr_BESS_Fuelx2', 
                    '8_Hr_BESS_Fuelx2', '10_Hr_BESS_Fuelx2'],
    models= ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
)

In [46]:
create_latex_tables_from_dict(percentage_profits_dic,  latex_path, 'table_percentage_profits', float_format='%.2f')

In [57]:
percentage_profits_dic

{'Thermal_Base':                                    Resource           CEM        PF  \
 0        NG 2-on-1 Combined Cycle (F-Frame) -1.980829e-14  3.889097   
 1           NG Combustion Turbine (F-Frame)  4.491871e-15  3.547685   
 2  Land-Based Wind - Class 1 - Technology 1  2.136037e-13 -1.688470   
 3                      Utility PV - Class 1  1.355000e-14 -0.048739   
 
       DLAC-p     DLAC-i       SLAC  
 0 -11.808143  22.950153 -32.438958  
 1   2.011166  26.779042  -0.740924  
 2  -5.190147  -1.817674 -29.204357  
 3  -3.399414   2.527780  -9.961167  ,
 '2_Hr_BESS':                                    Resource           CEM         PF  \
 0        NG 2-on-1 Combined Cycle (F-Frame) -6.521138e-13   1.483783   
 1           NG Combustion Turbine (F-Frame)  1.100332e-14   0.420550   
 2  Land-Based Wind - Class 1 - Technology 1  1.979060e-13  -1.598977   
 3                      Utility PV - Class 1 -2.441870e-13   2.090002   
 4       Utility-Scale Battery Storage - 2Hr -1.48461

In [47]:
# models2print = ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
models2print = ['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']
create_pmr_cpr_latex_table_all_cases(percentage_profits_dic, consumer_payments_change_df, latex_path, 'table3-5_pmr_cpr_all_cases', models=models2print)

In [48]:
# print 4 hour battery case percentage profits to csv
percentage_profits_dic['4_Hr_BESS'].to_csv(csv_path + '4_hour_battery_case_percentage_profits.csv', index=False)
percentage_profits_dic['4_Hr_BESS_Fuelx4'].to_csv(csv_path + '4_hour_battery_case_fuelx4_percentage_profits.csv', index=False)

In [49]:
battery_df = pd.DataFrame()
for key in percentage_profits_dic:
    print(f"Key: {key}")
    # print(percentage_profits_dic[key])
    battery_row = percentage_profits_dic[key][percentage_profits_dic[key]['Resource'] == 'Battery']
    battery_row.insert(0, 'Case', key)
    battery_df = pd.concat([battery_df, battery_row], ignore_index=True)
    # if battery_df.empty:
    #     battery_df = None
    # print(battery_df)

battery_df.to_csv(csv_path + 'battery_percentage_profits.csv', index=False)

Key: Thermal_Base
Key: 2_Hr_BESS
Key: 2_Hr_BESS_Fuelx2
Key: 4_Hr_BESS
Key: 4_Hr_BESS_Fuelx2
Key: 4_Hr_BESS_Fuelx3
Key: 4_Hr_BESS_Fuelx4
Key: 6_Hr_BESS
Key: 6_Hr_BESS_Fuelx2
Key: 8_Hr_BESS
Key: 8_Hr_BESS_Fuelx2
Key: 10_Hr_BESS
Key: 10_Hr_BESS_Fuelx2


In [50]:
battery_df

,Case,Resource,CEM,PF,DLAC-p,DLAC-i,SLAC


In [51]:
# # Convert the percentage profits for 'Thermal_Case' to a DataFrame
# thermal_case_df = percentage_profits_dic['Thermal_Case']

# # Define the filename
# thermal_case_csv_filename = pdf_path + 'thermal_case_percentage_profits.csv'

# # Save the DataFrame to a CSV file
# thermal_case_df.to_csv(thermal_case_csv_filename, index=False)

In [52]:
consumer_payments_dic

{'Thermal_Base': {'CEM': 17976722256.026386,
  'PF': 18178977221.301075,
  'DLAC-p': 17400670364.415073,
  'DLAC-i': 19082029511.353024,
  'SLAC': 15655521187.487644},
 '2_Hr_BESS': {'CEM': 21799400075.21762,
  'PF': 21979759414.1722,
  'DLAC-p': 21406277985.960793,
  'DLAC-i': 20502085957.560596,
  'SLAC': 19830365531.724724},
 '2_Hr_BESS_Fuelx2': {'CEM': 25122485031.702953,
  'PF': 28833195149.34454,
  'DLAC-p': 23296328002.790737,
  'DLAC-i': 23425738078.226295,
  'SLAC': 23451898129.18302},
 '4_Hr_BESS': {'CEM': 21761463158.397884,
  'PF': 22595114288.856853,
  'DLAC-p': 22126041023.333862,
  'DLAC-i': 20982261654.198483,
  'SLAC': 20676884741.182873},
 '4_Hr_BESS_Fuelx2': {'CEM': 24946681420.205677,
  'PF': 28854602698.541782,
  'DLAC-p': 24422457543.426056,
  'DLAC-i': 23600151897.217323,
  'SLAC': 23804558991.411278},
 '4_Hr_BESS_Fuelx3': {'CEM': 26738710009.47481,
  'PF': 27206603874.153275,
  'DLAC-p': 26229358046.5628,
  'DLAC-i': 25901057018.18066,
  'SLAC': 24483757247.9451

In [53]:

total_system_costs_fln =  'total_system_costs_' + date + '.pdf'

# Create a PDF file
with PdfPages(pdf_path + total_system_costs_fln) as pdf:

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_system_costs_df.values, colLabels=total_system_costs_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title(f'Total Costs for each case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)
# END: Add unit profit tables to PDF

In [54]:
total_nse_df.columns

Index(['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC'], dtype='object')

In [55]:
total_nse_fln =  'total_nse_' + date + '.pdf'

# Create a PDF file
with PdfPages(pdf_path + total_nse_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_nse_df.values, colLabels=total_nse_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Non-served Energy for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

In [56]:
total_unmet_rsv_fln =  'total_unmet_rsv_' + date + '.pdf'

# Create a PDF file
with PdfPages(pdf_path + total_unmet_rsv_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_unmet_rsv_df.values, colLabels=total_unmet_rsv_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Unmet Reserves for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)